In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import zipfile

#hello :]

# Unzip the data folder first

In [137]:
#hc - housing core
#bls - bureal of labor
#ctr - corporate tax rate
#rm -  rent median
#cs - census_starter
#fa - food access
#ue - unemployment
#me - medicare enrollment

df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv')
df_bls = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.xlsx')
df_ctr = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Corporate_tax_rate.csv')
df_rm_FY19 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2019_50_County_rev.xlsx')
df_rm_FY20 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2020_50_County_rev.xlsx')
df_rm_FY21 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2021_50_County.xlsx')
df_rm_FY22 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2022_FMR_50_county_rev.xlsx')
df_rm_FY23 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2023_FMR_50_county.xlsx')
df_fa = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FoodAccessResearchAtlasData2019.xlsx')
df_ue = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\Unemployment.xlsx', skiprows=4) #Bianca
df_me = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Medicare_Monthly_Enrollment_September_2022.csv')


df_cs = pd.read_csv('..\Data\data_zip\census_starter.csv')
df_test = pd.read_csv(r'..\Data\data_zip\test.csv')
df_train = pd.read_csv(r'..\Data\data_zip\train.csv')

C:\Users\jacks\AppData\Local\Temp\ipykernel_7316\858484407.py:11: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv')


## Format Census Starter to join into train and test data

In [72]:
df_cs
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()

variable,cfips,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers
0,1001,2017,55317.0,76.6,14.5,2.1,1.3
1,1001,2018,58786.0,78.9,15.9,2.0,1.1
2,1001,2019,58731.0,80.6,16.1,2.3,0.7
3,1001,2020,57982.0,82.7,16.7,2.3,0.6
4,1001,2021,62660.0,85.5,16.4,2.1,1.1
...,...,...,...,...,...,...,...
15705,56045,2017,59605.0,71.1,14.1,3.8,0.6
15706,56045,2018,52867.0,73.3,13.5,4.1,0.6
15707,56045,2019,57031.0,76.8,13.4,1.7,0.0
15708,56045,2020,53333.0,79.7,12.7,2.3,0.0


In [41]:
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
df_cs2.sort_values(['cfips']).reset_index(drop=True)
df_cs2 = pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_cs2

variable,cfips,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers
0,1001,2017,55317.0,76.6,14.5,2.1,1.3
1,1001,2018,58786.0,78.9,15.9,2.0,1.1
2,1001,2019,58731.0,80.6,16.1,2.3,0.7
3,1001,2020,57982.0,82.7,16.7,2.3,0.6
4,1001,2021,62660.0,85.5,16.4,2.1,1.1
...,...,...,...,...,...,...,...
15705,56045,2017,59605.0,71.1,14.1,3.8,0.6
15706,56045,2018,52867.0,73.3,13.5,4.1,0.6
15707,56045,2019,57031.0,76.8,13.4,1.7,0.0
15708,56045,2020,53333.0,79.7,12.7,2.3,0.0


### df_hc housing core looks like it is already in a joinable format

## Format bureal of labor to join into train and test data

In [116]:
df_bls2 = pd.pivot_table(df_bls, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2

measure_type,cfips,first_day_of_month,Employment,Labor force,Unemployment,Unemployment rate
0,1001,2011-01-01,23440.0,25800.0,2360.0,9.1
1,1001,2011-02-01,23516.0,25850.0,2334.0,9.0
2,1001,2011-03-01,23643.0,25862.0,2219.0,8.6
3,1001,2011-04-01,23787.0,25872.0,2085.0,8.1
4,1001,2011-05-01,23773.0,25863.0,2090.0,8.1
...,...,...,...,...,...,...
87646,17045,2022-06-01,NaN,NaN,318.0,3.6
87647,17045,2022-07-01,NaN,NaN,352.0,3.8
87648,17045,2022-08-01,NaN,NaN,327.0,3.6
87649,17045,2022-09-01,NaN,NaN,281.0,3.2


In [49]:
df_ctr2 = pd.melt(df_ctr, id_vars=['State'])
df_ctr2

,State,variable,value
0,Alabama,2008,4.23%
1,Alaska,2008,9.40%
2,Arizona,2008,6.97%
3,Arkansas,2008,6.50%
4,California,2008,8.84%
...,...,...,...
745,Virginia,2022,7.63%
746,Washington,2022,6.81%
747,West Virginia,2022,6.50%
748,Wisconsin,2022,7.90%


In [61]:
df_me
#No formatting change needed

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES
0,2022,April,National,US,National,Total,NaN,64665615,35102417,29563198,...,56614798,7752840,236979,7515861,58785851,29255529,29530322,50013581,23367558,26646023
1,2022,April,State,AL,Alabama,Total,1.0,1078349,495600,582749,...,876402,196610,5000,191610,998869,416218,582651,822177,268695,553482
2,2022,April,County,AL,Alabama,Autauga,1001.0,11492,5150,6342,...,9099,2339,55,2284,10643,4303,6340,7985,1829,6156
3,2022,April,County,AL,Alabama,Baldwin,1003.0,58902,27624,31278,...,51904,6798,131,6667,55097,23830,31267,44841,15131,29710
4,2022,April,County,AL,Alabama,Barbour,1005.0,6260,2440,3820,...,4877,1329,41,1288,5932,2113,3819,5050,1396,3654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,2021,September,County,VI,Virgin Islands,St. John,78020.0,921,745,176,...,*,47,*,*,838,662,176,470,299,171
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030.0,9625,6669,2956,...,8883,669,50,619,8675,5719,2956,5792,2859,2933
60038,2021,September,County,VI,Virgin Islands,Unknown,78999.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*
60039,2021,September,State,UK,Unknown,Unknown,99.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [148]:
df_ue2 = pd.melt(df_ue, id_vars=['FIPS_code', 'State', 'Area_name'])
df_ue2['Year'] = df_ue2['variable'].str[-4:]
df_ue2['variable'] = df_ue2['variable'].str[:-5]
df_ue2 = pd.pivot_table(df_ue2, values='value', index=['FIPS_code', 'State', 'Area_name', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_ue2['Year'].max()

'2021'

Unemployment data on a two year lag. Match the 2021 data with 2023 year, 2020 with 2022 and so on

In [73]:
df_me

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES
0,2022,April,National,US,National,Total,NaN,64665615,35102417,29563198,...,56614798,7752840,236979,7515861,58785851,29255529,29530322,50013581,23367558,26646023
1,2022,April,State,AL,Alabama,Total,1.0,1078349,495600,582749,...,876402,196610,5000,191610,998869,416218,582651,822177,268695,553482
2,2022,April,County,AL,Alabama,Autauga,1001.0,11492,5150,6342,...,9099,2339,55,2284,10643,4303,6340,7985,1829,6156
3,2022,April,County,AL,Alabama,Baldwin,1003.0,58902,27624,31278,...,51904,6798,131,6667,55097,23830,31267,44841,15131,29710
4,2022,April,County,AL,Alabama,Barbour,1005.0,6260,2440,3820,...,4877,1329,41,1288,5932,2113,3819,5050,1396,3654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,2021,September,County,VI,Virgin Islands,St. John,78020.0,921,745,176,...,*,47,*,*,838,662,176,470,299,171
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030.0,9625,6669,2956,...,8883,669,50,619,8675,5719,2956,5792,2859,2933
60038,2021,September,County,VI,Virgin Islands,Unknown,78999.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*
60039,2021,September,State,UK,Unknown,Unknown,99.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [74]:
df_train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [117]:
df_train['first_day_of_month'] = pd.to_datetime(df_train['first_day_of_month'])

In [96]:
df_bls2['first_day_of_month'] = df_bls2['first_day_of_month'].astype('object')
df_bls2

measure_type,cfips,first_day_of_month,Employment,Labor force,Unemployment,Unemployment rate
0,1001,2011-01-01,23440.0,25800.0,2360.0,9.1
1,1001,2011-02-01,23516.0,25850.0,2334.0,9.0
2,1001,2011-03-01,23643.0,25862.0,2219.0,8.6
3,1001,2011-04-01,23787.0,25872.0,2085.0,8.1
4,1001,2011-05-01,23773.0,25863.0,2090.0,8.1
...,...,...,...,...,...,...
87646,17045,2022-06-01,NaN,NaN,318.0,3.6
87647,17045,2022-07-01,NaN,NaN,352.0,3.8
87648,17045,2022-08-01,NaN,NaN,327.0,3.6
87649,17045,2022-09-01,NaN,NaN,281.0,3.2


In [118]:
print(df_bls2.dtypes)
print(df_train.dtypes)

measure_type
cfips                          int64
first_day_of_month    datetime64[ns]
Employment                   float64
Labor force                  float64
Unemployment                 float64
Unemployment rate            float64
dtype: object
row_id                           object
cfips                             int64
county                           object
state                            object
first_day_of_month       datetime64[ns]
microbusiness_density           float64
active                            int64
dtype: object


# Delete data_zip folder to load to Github

In [129]:
df_train_bls = df_train.merge(df_bls2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month'], indicator=True)

In [130]:
df_bls2['first_day_of_month'].max()

Timestamp('2022-11-01 00:00:00')

In [131]:
df_train_bls

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,Employment,Labor force,Unemployment,Unemployment rate,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,25944.0,26729.0,785.0,2.9,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,25873.0,26562.0,689.0,2.6,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,26052.0,26720.0,668.0,2.5,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,25969.0,26619.0,650.0,2.4,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,25992.0,26674.0,682.0,2.6,both
...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaN,NaN,NaN,NaN,left_only
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaN,NaN,NaN,NaN,left_only
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaN,NaN,NaN,NaN,left_only
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaN,NaN,NaN,NaN,left_only


In [105]:
df_train_bls[df_train_bls['Employment'].notnull()]

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,Employment,Labor force,Unemployment,Unemployment rate,_merge
122265,NaN,1001,NaN,NaN,2011-01-01,NaN,NaN,23440.0,25800.0,2360.0,9.1,right_only
122266,NaN,1001,NaN,NaN,2011-02-01,NaN,NaN,23516.0,25850.0,2334.0,9.0,right_only
122267,NaN,1001,NaN,NaN,2011-03-01,NaN,NaN,23643.0,25862.0,2219.0,8.6,right_only
122268,NaN,1001,NaN,NaN,2011-04-01,NaN,NaN,23787.0,25872.0,2085.0,8.1,right_only
122269,NaN,1001,NaN,NaN,2011-05-01,NaN,NaN,23773.0,25863.0,2090.0,8.1,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...
209769,NaN,17043,NaN,NaN,2022-06-01,NaN,NaN,502201.0,520242.0,18041.0,3.5,right_only
209770,NaN,17043,NaN,NaN,2022-07-01,NaN,NaN,497735.0,515679.0,17944.0,3.5,right_only
209771,NaN,17043,NaN,NaN,2022-08-01,NaN,NaN,498146.0,516156.0,18010.0,3.5,right_only
209772,NaN,17043,NaN,NaN,2022-09-01,NaN,NaN,494942.0,511587.0,16645.0,3.3,right_only


In [133]:
df_test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01
